In [107]:
import gensim
import gensim.downloader as api
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from langdetect import detect

In [108]:
word2vec_model = api.load('fasttext-wiki-news-subwords-300')


In [109]:
stop_words = stopwords.words('english')

def preprocess(sentence): 
    return [w for w in sentence.lower().split() if w not in stop_words]

def compose_overview(array_of_string):
    ov = ''
    for word in array_of_string: 
        if word != '': 
            ov += word + ' '
    return ov

def process_genres(string_genres):
    genres_list = [] 
    for i in range(2, len(string_genres)):
        if string_genres[i-3:i] == ': \'':
            gen = ''
            j = i
            while (string_genres[j] != '\''): 
                gen += string_genres[j]
                j += 1
            genres_list.append(gen.lower())
    return genres_list

In [110]:
movies_file = 'movies_metadata.csv'
movies = pd.read_csv(movies_file)
movies = movies[movies['status'] == 'Released']
movies = movies.dropna(subset=['overview'])

C:\Users\nikop\AppData\Local\Temp\ipykernel_8564\1364657094.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(movies_file)


In [111]:
genres = movies['genres']
genres_new = [process_genres(genre) for genre in movies['genres']]
movies['genres'] = genres_new
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[romance, comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[comedy, drama, romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### GENRE-BASED FILTERING

In [7]:
genres = []
for i in range(3):
    genre = input('write a genre to search for [Max 3], write | OK | to end\n')
    if genre != '':
        genres.append(genre)
    else: 
        break
if len(genres) != 0: 
    genres = set(genres)
    intersection = [len(list(set(x) & genres)) for x in movies['genres']]
    bool_mask = [x > 0 for x in intersection]
    movies = movies[bool_mask]

In [8]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[romance, comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[comedy, drama, romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### DATE-BASED FILTERING

In [75]:
date = input('write the year from which you want to search a movies from')
if date != '': 
    date += '-01-01'
    movies = movies[movies['release_date'] > date]
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1081,False,NaN,0,"[comedy, horror]",NaN,141210,tt2250194,en,The Sleepover,"The town of Derry has a secret, but no one tol...",...,2013-10-12,0.0,6.0,[],Released,NaN,The Sleepover,False,8.0,1.0
2323,False,NaN,12000000,"[comedy, drama, war]",NaN,10368,tt0120857,en,Tea with Mussolini,Semi-autobiographical film directed by Franco ...,...,1999-03-25,21001563.0,117.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Tea with Mussolini,False,5.8,32.0
2334,False,NaN,0,"[comedy, drama, romance]",NaN,14709,tt0139699,en,Varsity Blues,"In small-town Texas, high school football is a...",...,1999-01-15,0.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It takes a hero to know what's worth winning.,Varsity Blues,False,6.0,126.0
2372,False,NaN,10000000,"[comedy, romance]",NaN,10314,tt0160862,en,She's All That,High school hotshot Zach Siler is the envy of ...,...,1999-01-29,103166989.0,95.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,These two opposites attract... but EVERYONE'S ...,She's All That,False,5.7,425.0
2373,False,NaN,0,"[comedy, romance]",NaN,125548,tt0138279,en,The 24 Hour Woman,Grace tries to be the perfect mother and TV pr...,...,1999-01-29,0.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A story about getting everything you want and ...,The 24 Hour Woman,False,2.0,1.0


### VOTE-BASED FILTERING   

In [78]:
vote = input('write the minimum vote you want to search a movie for [don\'t be too bad]')
if vote != '':
    vote = float(vote)
    movies = movies[movies['vote_average'] > vote]
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
5,False,NaN,60000000,"[action, crime, drama, thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
12,False,"{'id': 117693, 'name': 'Balto Collection', 'po...",0,"[family, animation, adventure]",NaN,21032,tt0112453,en,Balto,An outcast half-wolf risks his life to prevent...,...,1995-12-22,11348324.0,78.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Part Dog. Part Wolf. All Hero.,Balto,False,7.1,423.0
13,False,NaN,44000000,"[history, drama]",NaN,10858,tt0113987,en,Nixon,An all-star cast powers this epic look at Amer...,...,1995-12-22,13681765.0,192.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Triumphant in Victory, Bitter in Defeat. He Ch...",Nixon,False,7.1,72.0
15,False,NaN,52000000,"[drama, crime]",NaN,524,tt0112641,en,Casino,The life of the gambling paradise – Las Vegas ...,...,1995-11-22,116112375.0,178.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,No one stays at the top forever.,Casino,False,7.8,1343.0


### DURATION-BASED FILTERING

In [68]:
duration = input('write the average duration of the movie you want to watch')
if duration != '':
    duration = float(duration)
    movies = movies[(movies['runtime'] > duration - 10) & (movies['runtime'] < duration + 10)]
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[animation, comedy, family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[adventure, fantasy, family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[romance, comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[comedy, drama, romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [135]:
overview = input("write a brief summary of what do you want to see")
written_lang = detect(overview)
overview = preprocess(overview)
display(overview)

['space', 'battle', 'aliens', 'fighting', 'humans']

In [ ]:
way_of_representation = input('do you want to search through the title or the overview? [T/O]')
while (True):
    if way_of_representation == 'O':
        by_title = False
        overview = input('write the overview of the film you want to search a similar one for')
        overview = preprocess(overview)
        display(overview)
        break
    elif way_of_representation == 'T':
        title = input('write the title of the movie you want to search a similar one for')
        by_title = True
        try: 
            overview = name_overview[name_overview['original_title'] == title]['overview']
            display(title)
            break
        except Exception:
            display(title + ' does not exist in the database, check for its correctness or try another title' )
    else: 
        way_of_representation = input('Sorry character not understood!\nPlease, write T if you want to search by Title, O if you want to search by Overview')

In [129]:
reduced_dataset = movies[['original_title','overview','genres','release_date','runtime','vote_average']]
length_overview = [len(x) for x in reduced_dataset['overview']]
reduced_dataset['length_overview'] = length_overview
reduced_dataset = reduced_dataset[reduced_dataset['length_overview'] > 100]

C:\Users\nikop\AppData\Local\Temp\ipykernel_8564\2922594896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_dataset['length_overview'] = length_overview


In [136]:
distances = reduced_dataset['overview'].apply(lambda ov: word2vec_model.wmdistance(overview, preprocess(ov)))

In [137]:
best_indeces = np.argpartition(distances, 5)[:5]
reduced_dataset['distances'] = distances
most_similar = reduced_dataset.iloc[best_indeces].sort_values(by=['distances'])
most_similar

,original_title,overview,genres,release_date,runtime,vote_average,length_overview,distances
29409,Survivor,During their search for a habitable planet the...,"[science fiction, action, fantasy]",2014-07-22,93.0,4.9,170,0.773635
18721,The Ledge,A thriller in which a battle of philosophies b...,"[drama, thriller]",2011-05-06,101.0,5.9,135,0.789202
39972,Invaders from the Deep,When aquatic aliens plot to take over the plan...,"[animation, science fiction]",1981-01-01,0.0,0.0,120,0.815548
10124,Gojira: Fainaru uôzu,Evil Space Aliens called the Xilians unleashes...,"[action, adventure, science fiction, fantasy, ...",2004-12-04,125.0,6.4,322,0.816845
23711,The Beast with a Million Eyes,An alien space craft lands in the desert. The ...,"[horror, science fiction]",1958-06-15,78.0,2.9,187,0.830859
